# Introduction

In this project, I will show a trick of using NumPy to solve a classical conditional probability problem of random walks: 
> Give an standard random walk (start at origin and with the probability of moving toward each direction 1/2 and 1/2), what is the probability of reaching 10 before reaching -20?

My Physics PhD friend mentioned this question to me as his interview question at Renaissance (a top quantitative trading company). I was interested by it and came up with the following trick. Since I never see others using this trick, I feel I have the responsibility to share it on GitHub so that it could also benefit others.

# Simulating Random Walks

First, we need to simulate random walks. For conveniency, let's create 50000 independent 1000-step random walks.

In [ ]:
import numpy as np

In [ ]:
nwalks = 50000
nsteps = 1000

In [ ]:
draws = np.random.randint(0, 2, size=(nwalks, nsteps))
steps = np.where(draws > 0, 1, -1)
steps

array([[-1, -1,  1, ..., -1, -1,  1],
       [ 1, -1,  1, ..., -1, -1,  1],
       [ 1,  1,  1, ...,  1, -1,  1],
       ...,
       [-1,  1,  1, ...,  1,  1,  1],
       [-1, -1, -1, ..., -1,  1, -1],
       [-1,  1, -1, ...,  1, -1, -1]])

Once we create those random steps, we can just use the `cumsum` function to create the walks (shows the positions over each step).

In [ ]:
walks = steps.cumsum(1)
walks

array([[ -1,  -2,  -1, ...,  52,  53,  52],
       [  1,   0,  -1, ...,  -8,  -9, -10],
       [ -1,   0,  -1, ...,   2,   1,   0],
       ...,
       [ -1,   0,  -1, ..., -34, -33, -32],
       [  1,   2,   3, ...,  34,  33,  34],
       [ -1,  -2,  -3, ...,  38,  37,  36]])

# Calculation Trick

We mark all the positions to the right of 10 as True, and all others as False.

In [ ]:
above_10 = (walks > 10)
above_10

array([[False, False, False, ...,  True,  True,  True],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True]])

Then we mark all the positions to the left of -20 as True, and all others as False.

In [ ]:
below_neg_20 = (walks < -20)
below_neg_20

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

Then we use the `cumsum` function again, since 'True' is equivalent with 1 in NumPy. When we `cumsum` 'True's, we are just `cumsum` 1s. Therefore we can mark all the positions to the right and include the first time reaching 10 as 'True'.

In [ ]:
pass_10 = (above_10.cumsum(1) > 1)
pass_10

array([[False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True]])

Similarly, we can mark all the positions to the right and including the first time reaching -20 as 'True'.

In [ ]:
pass_neg_20 = (below_neg_20.cumsum(1) > 1)
pass_neg_20

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ...,  True,  True,  True],
       ...,
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

Before we reach 10 or -20, all the positions should all be marked with 'False'.

If the we reach 10 for the first time before reaching -20 at certain postion, `pass_10` must be 'True' and `pass_neg_20` must be 'False' at this position. **Therefore `(pass_10 & ~pass_neg_20)` must be 'True' at this position and all positions afterward.** In other words, some positions in this random walk (row) are marked with 'True'.

If we reach -20 for the first time before reaching 10 at certain position, `pass_10` must be 'False' and `pass_neg_20` must be 'True' at this position. **Therefore `(pass_10 & ~pass_neg_20)` must be 'False' at this position and all positions afterward.** Notice that `(pass_10 & ~pass_neg_20)` are all 'False' before this position too. In other words, all positions in this single random walk (row) are marked with 'False' (with no 'True' at all).



In [ ]:
(pass_10 & ~pass_neg_20)

array([[False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True]])

Lastly, we just count how many rows have obtained the 'True' labels. They are random walks which reach 10 before -20. All other rows with no 'True' label have either never reached or reached 10 after -20.

In [ ]:
pass_10_before_pass_neg_20 = (pass_10 & ~pass_neg_20).any(1)
pass_10_before_pass_neg_20

array([ True,  True,  True, ...,  True,  True,  True])

Since we are doing a large number of random simulations (50000), to get the conditional probability of a random walks reaching 10 before -20, we just need to divide number of random walks which are labelled with 'True' by the total number of simulations. 

It is approximately 0.64512.

In [ ]:
pass_10_before_pass_neg_20.sum() / nwalks

0.64512

# Final Note

The above technique can be adapted with solve many other  time-series conditional probability questions. The general steps are:


- Create a large number of independent simulations.
- Within each simulation, mark the first time step (and afterward) that the condition #1 is satisfied as 'True'. We get a vector A of 'True's and 'False's.
- Within each simulation, mark the first time step (and afterward) that the condition #2 is satisfied as 'True'. We get a vector B of 'True's and 'False's.
- A & (negation of B) will have at least one 'True' if the condition #1 is reached before reaching condition #2. It will all be 'False' otherwise.
- Count the number of vectors which pass the test in the above step and divide this number of the total number of simulations is our wanted conditional probability.
